# **'Precio Media Anual'** Propiedades

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "tulancingo" in nombre
    and ('financiamientos' not in nombre and 'tiempo' not in nombre and 'poblacion' not in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
        # Renombrar la columna
    df.rename(columns={'Recamaras':'recamaras','Estacionamiento':'estacionamiento','m_total': 'metros_total','m_construido':'metros_construido','tiempo_publicacion':'tiempo_de_publicacion','Estacionamientos':'estacionamiento','Recamaras': 'recamaras', 
       'baño':'Banos','medio_baño':'Medio_banos', 'Seguridad':'seguridad','seguridad_privada':'seguridad','baño_total':'Banos_Total', 'cp':'CP',                  'Colonia':'colonia','Estado':'estado','Tipo':'tipo','Categoria':'categoria','Precio':'precio','m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)

In [16]:
import re

# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_resultados = pd.DataFrame(columns=['categoria', 'precio', 'mes', 'año'])
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Añadir las columnas de mes y año al DataFrame actual
        df['mes'] = mes
        df['año'] = ano
        df_resultados = pd.concat([df_resultados, df], ignore_index=True)
df_resultados[['categoria','precio','mes','año']]


,categoria,precio,mes,año
0,,8000000,4,2024
1,S1,4810000,4,2024
2,S1,4500000,4,2024
3,S1,4300000,4,2024
4,A3,3946465,4,2024
...,...,...,...,...
1883,B1,2890000,9,2023
1884,S,6100000,9,2023
1885,A1,3600000,9,2023
1886,B3,2480000,9,2023


# Regresión mensual (RF)

In [5]:
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,902500.0
1,3,2023,1045000.0
2,3,2024,995000.0
3,4,2024,955000.0
4,5,2022,1420000.0
5,5,2023,1200000.0
6,5,2024,650284.0
7,6,2023,1500000.0
8,6,2024,995000.0
9,7,2022,1365000.0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [7]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2022 = pd.DataFrame({'año': np.repeat(2022, 2),
                            'mes': range(11, 13)})
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 2),
                            'mes': range(11, 13)})
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 2),
                            'mes': range(1, 3)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2022,fechas_2023,fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2022,5,1420000.00
1,2022,7,1365000.00
2,2022,10,1350000.00
3,2022,11,1293800.00
4,2022,12,1293800.00
5,2023,3,1045000.00
6,2023,5,1200000.00
7,2023,6,1500000.00
8,2023,7,1149950.00
9,2023,9,1222500.00


In [8]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2022,5,1420000.00,mayo-2022
1,2022,7,1365000.00,julio-2022
2,2022,10,1350000.00,octubre-2022
3,2022,11,1293800.00,noviembre-2022
4,2022,12,1293800.00,diciembre-2022
5,2023,3,1045000.00,marzo-2023
6,2023,5,1200000.00,mayo-2023
7,2023,6,1500000.00,junio-2023
8,2023,7,1149950.00,julio-2023
9,2023,9,1222500.00,septiembre-2023


In [9]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[500000, 2000000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[500000, 1000000,1500000, 2000000, ],  # Definir los valores de las marcas en el eje y un poco antes del millón
        ticktext=['$500,000.00','$1,000,000.00','$1,500,000.00', '$2,000,000.00',],  # Definir el texto de las marcas en el eje y un poco antes del millón
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

# Regresión mensual (RF)
## Cortando 2023

In [11]:
df_resultados = df_resultados[~df_resultados['año'].isin([2022, 2023])]
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,902500.0
1,3,2024,995000.0
2,4,2024,955000.0
3,5,2024,650284.0
4,6,2024,995000.0
5,7,2024,841100.0
6,8,2024,882000.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [13]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 1),
                            'mes': range(1, 2)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2024,1,883990.16
1,2024,2,902500.00
2,2024,3,995000.00
3,2024,4,955000.00
4,2024,5,650284.00
5,2024,6,995000.00
6,2024,7,841100.00
7,2024,8,882000.00
8,2024,9,873353.84
9,2024,10,873353.84


In [14]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2024,1,883990.16,enero-2024
1,2024,2,902500.00,febrero-2024
2,2024,3,995000.00,marzo-2024
3,2024,4,955000.00,abril-2024
4,2024,5,650284.00,mayo-2024
5,2024,6,995000.00,junio-2024
6,2024,7,841100.00,julio-2024
7,2024,8,882000.00,agosto-2024
8,2024,9,873353.84,septiembre-2024
9,2024,10,873353.84,octubre-2024


In [15]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[500000, 1000000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[500000, 1000000,],  # Añadir la mitad entre 2000000 y 3000000
        ticktext=['$500,000.00','$1,000,000.00',],  # Añadir el texto para la marca de la mitad
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

# Regresión Anual

In [17]:
df_resultados['año'] = pd.to_numeric(df_resultados['año'], errors='coerce')
df_resultados['año'] = df_resultados['año'].fillna(0)
print(df_resultados['año'].dtype)

int64


In [18]:
precio_media_anual_pachuca = df_resultados.groupby('año')['precio'].median().reset_index()
precio_media_anual_pachuca.rename(columns={'precio': 'promedio'}, inplace=True)
precio_media_anual_pachuca

,año,promedio
0,2022,1370133.5
1,2023,1250000.0
2,2024,868000.0


In [19]:
# Utilizar pct_change para calcular el cambio porcentual entre últimos años
diferencia_porcentual = precio_media_anual_pachuca['promedio'].pct_change().iloc[1] * 100

# Proyecciones a los siguientes años, Comenzando con 2024
predicciones = pd.DataFrame()
predicciones[2024] = [precio_media_anual_pachuca[precio_media_anual_pachuca['año'] == 2024]['promedio'].iloc[0]]

# Proyectar los siguientes años aplicando la diferencia porcentual acumulativa
for año in range(2025, 2028):
    # Aplicar el crecimiento porcentual sobre el valor del año anterior
    predicciones[año] = round(predicciones[año-1] * (1 + diferencia_porcentual / 100), 2)
df_predicciones = pd.DataFrame(predicciones)
df_predicciones = df_predicciones.reset_index()
predict_pachuca = pd.melt(df_predicciones, id_vars='index', var_name='año', value_name='promedio')
predict_pachuca.drop('index', axis=1, inplace=True)
predict_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5600\2804342527.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,año,promedio
0,2024,868000.00
1,2025,791893.64
2,2026,722460.29
3,2027,659114.87


In [20]:
df_concatenado = pd.concat([precio_media_anual_pachuca, predict_pachuca], ignore_index=True)
df_concatenado['promedio'] = df_concatenado['promedio'].round()
df_concatenado

,año,promedio
0,2022,1370133.5
1,2023,1250000.0
2,2024,868000.0
3,2024,868000.0
4,2025,791893.64
5,2026,722460.29
6,2027,659114.87


In [21]:
# Datos
año = df_concatenado['año']
promedio = df_concatenado['promedio']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[0, 3000000],  # Establecer el rango del eje y
        tickvals=[0, 1000000,2000000, 3000000,],  # Definir los valores de las marcas en el eje y
        ticktext=['$0.00', '$1,000,000.00','$2,000,000.00','$3,000,000.00',],  # Definir el texto de las marcas en el eje y
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()